In [1]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import skimage
import cv2
import glob
import pandas as pd

from tensorflow.keras import layers
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Model
from tensorflow.keras.models import load_model #added
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, Dropout, UpSampling2D, LayerNormalization, concatenate
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from tensorflow.image import resize
from skimage import data, io,color
from skimage.io import imread_collection #loads a collection of images
from skimage.io import imread, imshow
from pathlib import Path

from sklearn.decomposition import PCA

import matplotlib as mpl
import matplotlib.pyplot as plt

In [2]:
org_path = 'C:/Study/Semester2/Machine Learning/smaller2/boy1.jpg'

In [3]:
#Preprocessing


from PIL import Image
import numpy as np
from skimage import color
import torch
import torch.nn.functional as F
#from IPython import embed # don't use yet

def load_img(img_path):
	out_np = np.asarray(Image.open(img_path))
	if(out_np.ndim==2):
		out_np = np.tile(out_np[:,:,None],3)
	return out_np

def resize_img(img, HW=(256,256), resample=3):
	return np.asarray(Image.fromarray(img).resize((HW[1],HW[0]), resample=resample))

def preprocess_img(img_rgb_orig, HW=(256,256), resample=3):
	# return original size L and resized L as torch Tensors
	img_rgb_rs = resize_img(img_rgb_orig, HW=HW, resample=resample)
	
	img_lab_orig = color.rgb2lab(img_rgb_orig)
	img_lab_rs = color.rgb2lab(img_rgb_rs)

	img_l_orig = img_lab_orig[:,:,0]
	img_ab_orig = img_lab_orig[:,:,1:]   
	img_l_rs = img_lab_rs[:,:,0]
	img_ab_rs = img_lab_rs[:,:,1:]
	#tens_orig_l = torch.Tensor(img_l_orig)[None,None,:,:]
	#tens_rs_l = torch.Tensor(img_l_rs)[None,None,:,:]
	tens_orig_l = tf.convert_to_tensor(img_l_orig)[None,None,:,:]
	tens_orig_ab = tf.convert_to_tensor(img_ab_orig)[None,None,:,:]
	tens_rs_l = tf.convert_to_tensor(img_l_rs)[None,None,:,:]
	tens_rs_ab = tf.convert_to_tensor(img_ab_rs)[None,None,:,:]  
	return tens_orig_l, tens_rs_l, tens_orig_ab,tens_rs_ab

def postprocess_tens(tens_orig_l, out_ab, mode='bilinear'):
	# tens_orig_l 	1 x 1 x H_orig x W_orig
	# out_ab 		1 x 2 x H x W

	HW_orig = tens_orig_l.shape[2:]
	print(HW_orig)
	HW = out_ab.shape[2:]
	print(HW)
	# call resize function if needed
	if(HW_orig[0]!=HW[0] or HW_orig[1]!=HW[1]):
		out_ab_orig = resize(out_ab, size=HW_orig) #F.interpolate(out_ab, size=HW_orig, mode='bilinear')
	else:
		out_ab_orig = out_ab

	out_lab_orig = concatenate((tens_orig_l, out_ab_orig)) #torch.cat((tens_orig_l, out_ab_orig), dim=1)
	return color.lab2rgb(out_lab_orig.data.cpu().numpy()[0,...].transpose((1,2,0)))

In [4]:
np_img_org = load_img(org_path)

In [5]:
print('original shape:',  np_img_org.shape, np.min(np_img_org), np.max(np_img_org))

original shape: (750, 600, 3) 0 255


In [6]:
tens_orig_l, tens_rs_l, tens_orig_ab,tens_rs_ab = preprocess_img(np_img_org, HW=(256,256), resample=3)

In [13]:
print(tens_orig_l.shape, tens_rs_l.shape, tens_orig_ab.shape,tens_rs_ab.shape)

(1, 1, 750, 600) (1, 1, 256, 256) (1, 1, 750, 600, 2) (1, 1, 256, 256, 2)


In [10]:
def mode(): 
    model2 = Sequential([
        #encoder
        Conv2D(128, 2, strides = 1, padding = "same", activation = "relu", input_shape= (1,256, 256)), #testing shape changes
        Conv2D(256, 2, strides = 1, padding = "same", activation = "relu"), 
        Conv2D(512, 2, strides = 1, padding = "same", activation = "relu"), #filter = 256
        
        #decoder
  
        Conv2D(256, 2, padding = "same", activation = "relu"), 
        
        Conv2D(128, 2, padding = "same", activation = "relu"), 

        Conv2D(256, 2, padding = "same", activation = "softmax"),

    ])
    model2.compile(optimizer ="adam", loss = "mse", metrics = ["accuracy"])
    ##LayerNormalization()
    return model2

In [11]:
model2 = mode() 
model2.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 1, 256, 128)       131200    
                                                                 
 conv2d_1 (Conv2D)           (None, 1, 256, 256)       131328    
                                                                 
 conv2d_2 (Conv2D)           (None, 1, 256, 512)       524800    
                                                                 
 conv2d_3 (Conv2D)           (None, 1, 256, 256)       524544    
                                                                 
 conv2d_4 (Conv2D)           (None, 1, 256, 128)       131200    
                                                                 
 conv2d_5 (Conv2D)           (None, 1, 256, 256)       131328    
                                                                 
Total params: 1,574,400
Trainable params: 1,574,400
Non-

In [12]:
epochs = 100
history = model2.fit(tens_rs_l, tens_rs_ab, epochs = epochs, verbose = 2)  # add validation_split = 0.2

Epoch 1/100


ValueError: in user code:

    File "C:\Users\midyr\anaconda3\lib\site-packages\keras\engine\training.py", line 1021, in train_function  *
        return step_function(self, iterator)
    File "C:\Users\midyr\anaconda3\lib\site-packages\keras\engine\training.py", line 1010, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "C:\Users\midyr\anaconda3\lib\site-packages\keras\engine\training.py", line 1000, in run_step  **
        outputs = model.train_step(data)
    File "C:\Users\midyr\anaconda3\lib\site-packages\keras\engine\training.py", line 860, in train_step
        loss = self.compute_loss(x, y, y_pred, sample_weight)
    File "C:\Users\midyr\anaconda3\lib\site-packages\keras\engine\training.py", line 918, in compute_loss
        return self.compiled_loss(
    File "C:\Users\midyr\anaconda3\lib\site-packages\keras\engine\compile_utils.py", line 201, in __call__
        loss_value = loss_obj(y_t, y_p, sample_weight=sw)
    File "C:

In [78]:
model2.save('Zhang-ours.h5')

In [8]:
#Load the model
savedModel=load_model('Zhang-ours.h5')
savedModel.summary()

Model: "sequential_11"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_66 (Conv2D)          (None, 1, 256, 1)         1048577   
                                                                 
 conv2d_67 (Conv2D)          (None, 1, 256, 64)        262208    
                                                                 
 conv2d_68 (Conv2D)          (None, 1, 256, 256)       65792     
                                                                 
 conv2d_69 (Conv2D)          (None, 1, 256, 256)       262400    
                                                                 
 conv2d_70 (Conv2D)          (None, 1, 256, 128)       131200    
                                                                 
 conv2d_71 (Conv2D)          (None, 1, 256, 2)         1026      
                                                                 
Total params: 1,771,203
Trainable params: 1,771,203
N

In [11]:
test_predict = savedModel.predict(tens_rs_l) #what shape does the prediction takes?

In [12]:
test_predict.shape

(1, 1, 256, 2)

In [52]:
tens_orig_l.shape[2:]

TensorShape([750, 600])

In [56]:
out_img_eccv16 = postprocess_tens(tens_orig_l, test_predict)

(750, 600)
(256, 2)


ValueError: A `Concatenate` layer requires inputs with matching shapes except for the concatenation axis. Received: input_shape=((1, 1, 750, 600), (1, 750, 600, 2))

In [16]:
# Model

#import torch
#import torch.nn as nn
import numpy as np
from IPython import embed

from .base_color import *

class ECCVGenerator(BaseColor):
    def __init__(self, norm_layer=nn.BatchNorm2d):
        super(ECCVGenerator, self).__init__()

        model1=[nn.Conv2d(1, 64, kernel_size=3, stride=1, padding=1, bias=True),]
        model1+=[nn.ReLU(True),]
        model1+=[nn.Conv2d(64, 64, kernel_size=3, stride=2, padding=1, bias=True),]
        model1+=[nn.ReLU(True),]
        model1+=[norm_layer(64),]

        model2=[nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1, bias=True),]
        model2+=[nn.ReLU(True),]
        model2+=[nn.Conv2d(128, 128, kernel_size=3, stride=2, padding=1, bias=True),]
        model2+=[nn.ReLU(True),]
        model2+=[norm_layer(128),]

        model3=[nn.Conv2d(128, 256, kernel_size=3, stride=1, padding=1, bias=True),]
        model3+=[nn.ReLU(True),]
        model3+=[nn.Conv2d(256, 256, kernel_size=3, stride=1, padding=1, bias=True),]
        model3+=[nn.ReLU(True),]
        model3+=[nn.Conv2d(256, 256, kernel_size=3, stride=2, padding=1, bias=True),]
        model3+=[nn.ReLU(True),]
        model3+=[norm_layer(256),]

        model4=[nn.Conv2d(256, 512, kernel_size=3, stride=1, padding=1, bias=True),]
        model4+=[nn.ReLU(True),]
        model4+=[nn.Conv2d(512, 512, kernel_size=3, stride=1, padding=1, bias=True),]
        model4+=[nn.ReLU(True),]
        model4+=[nn.Conv2d(512, 512, kernel_size=3, stride=1, padding=1, bias=True),]
        model4+=[nn.ReLU(True),]
        model4+=[norm_layer(512),]

        model5=[nn.Conv2d(512, 512, kernel_size=3, dilation=2, stride=1, padding=2, bias=True),]
        model5+=[nn.ReLU(True),]
        model5+=[nn.Conv2d(512, 512, kernel_size=3, dilation=2, stride=1, padding=2, bias=True),]
        model5+=[nn.ReLU(True),]
        model5+=[nn.Conv2d(512, 512, kernel_size=3, dilation=2, stride=1, padding=2, bias=True),]
        model5+=[nn.ReLU(True),]
        model5+=[norm_layer(512),]

        model6=[nn.Conv2d(512, 512, kernel_size=3, dilation=2, stride=1, padding=2, bias=True),]
        model6+=[nn.ReLU(True),]
        model6+=[nn.Conv2d(512, 512, kernel_size=3, dilation=2, stride=1, padding=2, bias=True),]
        model6+=[nn.ReLU(True),]
        model6+=[nn.Conv2d(512, 512, kernel_size=3, dilation=2, stride=1, padding=2, bias=True),]
        model6+=[nn.ReLU(True),]
        model6+=[norm_layer(512),]

        model7=[nn.Conv2d(512, 512, kernel_size=3, stride=1, padding=1, bias=True),]
        model7+=[nn.ReLU(True),]
        model7+=[nn.Conv2d(512, 512, kernel_size=3, stride=1, padding=1, bias=True),]
        model7+=[nn.ReLU(True),]
        model7+=[nn.Conv2d(512, 512, kernel_size=3, stride=1, padding=1, bias=True),]
        model7+=[nn.ReLU(True),]
        model7+=[norm_layer(512),]

        model8=[nn.ConvTranspose2d(512, 256, kernel_size=4, stride=2, padding=1, bias=True),]
        model8+=[nn.ReLU(True),]
        model8+=[nn.Conv2d(256, 256, kernel_size=3, stride=1, padding=1, bias=True),]
        model8+=[nn.ReLU(True),]
        model8+=[nn.Conv2d(256, 256, kernel_size=3, stride=1, padding=1, bias=True),]
        model8+=[nn.ReLU(True),]

        model8+=[nn.Conv2d(256, 313, kernel_size=1, stride=1, padding=0, bias=True),]

        self.model1 = nn.Sequential(*model1)
        self.model2 = nn.Sequential(*model2)
        self.model3 = nn.Sequential(*model3)
        self.model4 = nn.Sequential(*model4)
        self.model5 = nn.Sequential(*model5)
        self.model6 = nn.Sequential(*model6)
        self.model7 = nn.Sequential(*model7)
        self.model8 = nn.Sequential(*model8)

        self.softmax = nn.Softmax(dim=1)
        self.model_out = nn.Conv2d(313, 2, kernel_size=1, padding=0, dilation=1, stride=1, bias=False)
        self.upsample4 = nn.Upsample(scale_factor=4, mode='bilinear')

    def forward(self, input_l):
        conv1_2 = self.model1(self.normalize_l(input_l))
        conv2_2 = self.model2(conv1_2)
        conv3_3 = self.model3(conv2_2)
        conv4_3 = self.model4(conv3_3)
        conv5_3 = self.model5(conv4_3)
        conv6_3 = self.model6(conv5_3)
        conv7_3 = self.model7(conv6_3)
        conv8_3 = self.model8(conv7_3)
        out_reg = self.model_out(self.softmax(conv8_3))

        return self.unnormalize_ab(self.upsample4(out_reg))

def eccv16(pretrained=True):
	model = ECCVGenerator()
	if(pretrained):
		import torch.utils.model_zoo as model_zoo
		model.load_state_dict(model_zoo.load_url('https://colorizers.s3.us-east-2.amazonaws.com/colorization_release_v2-9b330a0b.pth',map_location='cpu',check_hash=True))
	return model

ImportError: attempted relative import with no known parent package